# Proyecto final

---

## Objetivo
---

Este proyecto tiene como objetivo desarrollar el análisis de una muestra de viajes de una de las empresas de servicios de transporte privado más reconocidas del mundo. Mediante dicho análisis, se busca preparar una Tabla Análitica de Datos (TAD) que permita la correcta implementación de modelos que predigan el precio ideal de un viaje para los usuarios que buscan transportarse de manera segura y con conductores confiables.

El dataset contiene las siguientes variables:

- **key**: identificador único para cada viaje.
- **fare_amount**: precio de cada viaje en dólares.
- **pickup_datetime**: fecha y hora en que se activó el medidor.
- **passenger_count**: el número de pasajeros en el vehículo (valor ingresado por el conductor).
- **pickup_longitude**: la geolocalización (longitud) en la que se activó el medidor.
- **pickup_latitude**: la geolocalización (latitud) en la que se activó el medidor.
- **dropoff_longitude**: la longitud en la que se desconectó el medidor.
- **dropoff_latitude**: la latitud donde se desconectó el medidor.


## Fuente
---
Kaggle.

## Datos
---
**Nombre:** Alan Ruiz Mondragón.  
**Grupo:** 18.

## Librerias
---

In [203]:
#Importamos librerias
from  functools import reduce
from  scipy.stats  import  normaltest
from category_encoders.count import CountEncoder
from plotly.offline import plot,iplot
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import ksone
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from varclushi import VarClusHi
import cufflinks as cf
import datetime
import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re 
import seaborn as sns
import unicodedata
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

#Definimos configuraciones
cf.go_offline()
pd.set_option("display.max_columns",200)
pd.set_option("display.max_rows",200)

## Funciones
---

In [204]:
#Definimos una función para limpieza de texto
def clean_text(text, pattern="[^a-zA-Z0-9]"):
    text=str(text)
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().lstrip().split())
    return cleaned_text if cleaned_text!="nan" else np.nan

#Definimos una función para revisar la completitud de nuestras variables
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

def unitarias(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")

def categoricas(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")
        result[col]=result[col].map(lambda x:str(round(x*100,2))+"%")
        result.reset_index(inplace=True)
        result.columns=[col+"_valores","%_aparicion"]
    return result

def bar(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="JetBrains Mono, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'JetBrains Mono, monospace',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'JetBrains Mono monospace',"size": 18,
        "color": '#000000'}}},title_font_family="JetBrains Mono, monospace",title_font_color="#000000",template="plotly_white")
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='bar',x="conteo",y=col,title=title,asFigure=True,barmode="overlay",sortbars=True,color="#000000",layout=layout)
    fig.update_layout(width=800)
    fig.update_traces(marker_color='#005a96')
    return fig

def pie(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="JetBrains Mono, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'JetBrains Mono, monospace',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'JetBrains Mono monospace',"size": 18,
        "color": '#000000'}}},title_font_family="JetBrains Mono, monospace",title_font_color="#000000",template="plotly_white")
    #layout = go.Layout(template="plotly_white")
    colors=[ "#581845", "#900c3f","#c70039","#ff5733","#ffc305","#005ba3","#0061a9","#1567af","#226cb6","#2c72bc", "#0061a9","#4c79b7","#7492c6","#98acd4","#bbc7e2","#dde3f1","#ffffff"
]
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='pie',labels="conteo",values=col,title=title,asFigure=True,theme="white")
    
    fig.update_traces(textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    return fig
    
def box(df,col,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='box',title=title,asFigure=True,theme="white",layout=layout,color="#005a96", boxpoints='outliers')
    return fig

def histogram(df,col,bins,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='histogram',x=col,bins=bins,title=title,asFigure=True,theme="white",layout=layout,color="#003e6c")
    fig.update_traces(opacity=0.90)
    return fig

def OUTLIERS(df,cols):
    results=pd.DataFrame()
    data_iqr=df.copy()
    data_per=df.copy()
    total=[]
    total_per=[]
    total_z=[]
    indices_=[]

    for col in cols:
        #IQR
        Q1=df[col].quantile(0.25)
        Q3=df[col].quantile(0.75)
        IQR=Q3-Q1
        INF=Q1-1.5*(IQR)
        SUP=Q3+1.5*(IQR)
    
        
        n_outliers=df[(df[col] < INF) | (df[col] > SUP)].shape[0]
        total.append(n_outliers)
        indices_iqr=list(df[(df[col] < INF) | (df[col] > SUP)].index)
        #data_iqr=data_iqr[~(data_iqr[col] < INF) | (data_iqr[col] > SUP)].reset_index(drop=True)
        
        #Percentiles
        INF_pe=np.percentile(df[col].dropna(),5)
    
        SUP_pe=np.percentile(df[col].dropna(),95)
        n_outliers_per=df[(df[col] < INF_pe) | (df[col] > SUP_pe)].shape[0]
        total_per.append(n_outliers_per)
        indices_per=list(df[(df[col] < INF_pe) | (df[col] > SUP_pe)].index)
        #data_per=data_per[~(data_per[col] < INF_pe) | (data_per[col] > SUP_pe)].reset_index(drop=True)
        
        #MEAN CHANGE
        
        #Obtenemos todos los percentiles además del máximo
        perc_100 = [x / 100 for x in range(100)]
        dist = df[col].describe(perc_100).iloc[4:]
        #Obtenemos el cambio entre percentiles
        change_dist = df[col].describe(perc_100).iloc[4:].diff()
        #Obtenemos el cambio promedio entre percentiles
        mean_change = df[col].describe(
            perc_100).iloc[4:].diff().mean()
        #Si el cambio entre el percentil 99 y el maximo es mayor a el cambio promedio entonces:
        if change_dist["max"] > mean_change:
            #La banda superior será el máximo menos el cambio promedio
            ub = dist["max"] - mean_change
            #si la banda superior es más pequeña que el percentil 99 , modificamos la banda para que tome el percentil 99
            if ub < dist["99%"]:
                ub = dist["99%"]
        else:
        #Si el cambio entre el percentil 99 y el maximo es menor o igual a el cambio promedio entonces se toma el percentil 99
            ub = dist["max"]

        if change_dist["1%"] > mean_change:
            lb = dist["0%"] + mean_change
            if lb > dist["1%"]:
                lb = dist["1%"]
        else:
            lb = dist["0%"]
        n_total_z=df[(df[col] < lb) | (df[col] > ub)].shape[0]
        total_z.append(n_total_z)
        indices_z=list(df[(df[col] < lb) | (df[col] > ub)].index)
        
        indices_.append(aux_outliers(indices_iqr,indices_per,indices_z))
        
    results["features"]=cols
    results["n_outliers_IQR"]=total
    results["n_outliers_Percentil"]=total_per
    results["n_outliers_Mean_Change"]=total_z
    results["n_outliers_IQR_%"]=round((results["n_outliers_IQR"]/df.shape[0])*100,2)
    results["n_outliers_Percentil_%"]=round((results["n_outliers_Percentil"]/df.shape[0])*100,2)
    results["n_outliers_Mean_Change_%"]=round((results["n_outliers_Mean_Change"]/df.shape[0])*100,2)
    results["indices"]=indices_
    results["total_outliers"]=results["indices"].map(lambda x:len(x))
    results["%_outliers"]=results["indices"].map(lambda x:round(((len(x)/df.shape[0])*100),2))
    results=results[['features', 'n_outliers_IQR', 'n_outliers_Percentil',
       'n_outliers_Mean_Change', 'n_outliers_IQR_%', 'n_outliers_Percentil_%',
       'n_outliers_Mean_Change_%',  'total_outliers', '%_outliers','indices']]
    return results

def missings_digit(x):
    if sum([y.isdigit() for y in str(x)])>0:
        result=np.nan
    else:
        result=x
    return result

def aux_outliers(a,b,c):
    a=set(a)
    b=set(b)
    c=set(c)
    
    a_=a.intersection(b)

    b_=b.intersection(c)

    c_=a.intersection(c)

    outliers_index=list(set(list(a_)+list(b_)+list(c_)))
    return outliers_index

def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    print(x_i)
    print(m_i)
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO (La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO (La porporción de categorias es la misma que la general)")

## Dataset
---

In [205]:
#Cargamos los datos
df = pd.read_csv('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/proyecto_final/data/uber.csv')

In [206]:
#Visualizamos el dataset
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [207]:
#Eliminamos la columna desconocida y la variable key debido a su inconsistencia de identificador y los datos presentes
df.drop(columns={'Unnamed: 0','key'}, axis=1, inplace=True)
#Visualizamos nuevamente el dataset
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [208]:
#Modificamos la variable de fecha a un tipo datetime para su posterior manipulación
#df.pickup_datetime = pd.to_datetime(df.pickup_datetime).dt.date

In [209]:
#Visualizamos el dataset
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


**Tamaño**

In [210]:
#Visualizamos el tamaño del dataset
df.shape

(200000, 7)

**Formato**

In [211]:
#Visualizamos la información del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fare_amount        200000 non-null  float64
 1   pickup_datetime    200000 non-null  object 
 2   pickup_longitude   200000 non-null  float64
 3   pickup_latitude    200000 non-null  float64
 4   dropoff_longitude  199999 non-null  float64
 5   dropoff_latitude   199999 non-null  float64
 6   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 10.7+ MB


**Diccionario**

In [212]:
#Cargamos el diccionario de datos del dataset principal
diccionario = pd.read_excel('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/proyecto_final/diccionario/diccionario.xlsx')

In [213]:
#Visualizamos el diccionario de datos del dataset principal 
diccionario

,variable_id,variable_nombre,variable_descripcion
0,1,key,identificador único para cada viaje
1,2,fare_amount,precio de cada viaje en dólares
2,3,pickup_datetime,fecha y hora en que se activó el medidor
3,4,passenger_count,el número de pasajeros en el vehículo (valor i...
4,5,pickup_longitude,la geolocalización (longitud) en la que se act...
5,6,pickup_latitude,la geolocalización (latitud) en la que se acti...
6,7,dropoff_longitude,la longitud en la que se desconectó el medidor
7,8,dropoff_latitude,la latitud donde se desconectó el medidor


## Calidad de datos
---

### Definición

La calidad de datos se refiere al grado en que los datos se ajustan a los criterios establecidos para su uso. Estos criterios de calidad de los datos abarcan aspectos como exactitud, coherencia, actualización, exhaustividad y decirigibilidad. La calidad de los datos es uno de los principales factores en el éxito de la base de datos. Si los datos no cumplen con los criterios de calidad adecuados, la información generada a partir de dichos datos también será maliciosa o inútil. Por lo tanto, es crítica la necesidad de vigilancia y mejora de la calidad de los datos.

### Etiquetado de variables

#### Contenido

Visualizamos el contenido y las columnas de los datasets nuevamente para registrar los siguientes pasos de calidad.

In [214]:
#Visualizamos el dataset
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [215]:
#Visualizamos las columnas contenidas dentro del dataset principal
df.columns

Index(['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count'],
      dtype='object')

In [216]:
#Visualizamos el contenido del datset principal
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fare_amount        200000 non-null  float64
 1   pickup_datetime    200000 non-null  object 
 2   pickup_longitude   200000 non-null  float64
 3   pickup_latitude    200000 non-null  float64
 4   dropoff_longitude  199999 non-null  float64
 5   dropoff_latitude   199999 non-null  float64
 6   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 10.7+ MB


In [217]:
#Visualizamos la distribución del contenido por tipos de datos de la columna del dataset principal 
df.dtypes.value_counts()

float64    5
object     1
int64      1
dtype: int64

##### Distribución del contenido

Los tipos de datos contenidos en el dataset se distribuyen de la siguiente manera

- Columnas de tipo float64: 5
- Columnas de tipo int64: 1
- Columnas de tipo object: 1

#### Prefijos

Realizamos el etiquetado de las variables utilizando los siguientes prefijos que utilizaremos para los diferentes tipos de variables:

- **c_**: Variables numericas (discretas y continuas).
- **v_**: Variables categoricas.
- **d_**: Variables tipo fecha.
- **t_**: Variables de texto (comentarios, descripciones, url, etc.).
- **g_**: Variables geograficas.

#### Etiquetado

**Nota**: Es importante mencionar que la variable target es la variable *****fare_amount*****, ya que el objetivo posterior al análisis es realizar un modelo de aprendizaje supervisado que nos permita predecir el precio de un viaje.

In [218]:
#Realizamos el etiquetado de las variables del datset principal de acuerdo a los prefijos anteriormente establecidos
c_feats=["fare_amount", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "passenger_count"] 
d_feats=["pickup_datetime", ]

In [219]:
#Aplicamos 
c_feats_new = ["c_" + x for x in c_feats]
d_feats_new = ["d_" + x for x in d_feats]

#Renombramos las columnas
df.rename(columns = dict(zip(c_feats, c_feats_new)), inplace = True)
df.rename(columns = dict(zip(d_feats, d_feats_new)), inplace = True)

In [220]:
#Validamos el etiquetado
df.columns

Index(['c_fare_amount', 'd_pickup_datetime', 'c_pickup_longitude',
       'c_pickup_latitude', 'c_dropoff_longitude', 'c_dropoff_latitude',
       'c_passenger_count'],
      dtype='object')

### Duplicados

#### General

In [221]:
#Total de registros duplicados de forma general
df.duplicated().sum()

0

In [222]:
df.loc[[28453,46687]]

,c_fare_amount,d_pickup_datetime,c_pickup_longitude,c_pickup_latitude,c_dropoff_longitude,c_dropoff_latitude,c_passenger_count
28453,3.7,2010-11-19 23:07:00 UTC,0.0,0.0,0.0,0.0,1
46687,3.7,2010-11-19 08:01:00 UTC,0.0,0.0,0.0,0.0,1


In [201]:
df[(df["c_fare_amount"] == 3.7) & (df["c_passenger_count"] == 1) & (df["c_pickup_latitude"] == 0.0)]

,c_fare_amount,d_pickup_datetime,c_pickup_longitude,c_pickup_latitude,c_dropoff_longitude,c_dropoff_latitude,c_passenger_count
9179,3.7,2012-07-18,0.0,0.0,0.000000,0.000000,1
20274,3.7,2010-01-29,0.0,0.0,0.000000,0.000000,1
28347,3.7,2010-07-24,0.0,0.0,0.000000,0.000000,1
28453,3.7,2010-11-19,0.0,0.0,0.000000,0.000000,1
32233,3.7,2010-02-04,0.0,0.0,0.000000,0.000000,1
46687,3.7,2010-11-19,0.0,0.0,0.000000,0.000000,1
46917,3.7,2009-03-21,0.0,0.0,0.000000,0.000000,1
47460,3.7,2011-09-14,0.0,0.0,0.000000,0.000000,1
49239,3.7,2009-05-09,0.0,0.0,0.000000,0.000000,1
52547,3.7,2010-11-22,0.0,0.0,0.000000,0.000000,1


In [199]:
df[df.duplicated()]

,c_fare_amount,d_pickup_datetime,c_pickup_longitude,c_pickup_latitude,c_dropoff_longitude,c_dropoff_latitude,c_passenger_count
46687,3.7,2010-11-19,0.0,0.0,0.0,0.0,1
48964,5.3,2011-11-03,0.0,0.0,0.0,0.0,1
57923,5.7,2010-06-20,0.0,0.0,0.0,0.0,1
59370,10.0,2014-01-16,0.0,0.0,0.0,0.0,1
67258,6.5,2010-05-10,0.0,0.0,0.0,0.0,2
69861,8.1,2010-12-17,0.0,0.0,0.0,0.0,1
81814,7.5,2015-03-26,0.0,0.0,0.0,0.0,1
90822,5.7,2011-10-11,0.0,0.0,0.0,0.0,1
92070,5.0,2014-07-13,0.0,0.0,0.0,0.0,1
101191,5.7,2012-06-18,0.0,0.0,0.0,0.0,1


In [145]:
df.shape[0]

200000

In [144]:
df.duplicated().sum()/df.shape[0]

0.000235

### Completitud

Se recomienda que las variables cuenten con una completitud de al menos 80%, de modo que las que no cumplan con esta condición serán eliminadas.

In [68]:
df.isnull().sum(0)

c_fare_amount          0
d_pickup_datetime      0
c_pickup_longitude     0
c_pickup_latitude      0
c_dropoff_longitude    1
c_dropoff_latitude     1
c_passenger_count      0
dtype: int64

In [69]:
#Revisamos la completitud de las columnas del dataset y observamos que aunque no hay peridda de valores menor al 80%, tenemos dos variables con un valor faltante
completitud(df)

,columna,total,completitud
0,c_dropoff_longitude,1,99.9995
1,c_dropoff_latitude,1,99.9995
2,c_fare_amount,0,100.0000
3,d_pickup_datetime,0,100.0000
4,c_pickup_longitude,0,100.0000
5,c_pickup_latitude,0,100.0000
6,c_passenger_count,0,100.0000


In [70]:
#Revisamos nuevamente el tamaño antes de eliminar las columnas con información incompleta 
df.shape

(200000, 7)

In [71]:
#Debido a que los datos faltantes representan un porcentaje bajo, es decir, menor al 1%, se decisió eliminar los registros con datos faltantes
df.dropna(inplace=True)

In [72]:
##Revisamos nuevamente el tamaño despúes de haber de eliminado las columnas con información incompleta 
df.shape

(199999, 7)

In [73]:
#Revisamos que no quedan columnas incompletas, es decir, con información menor del 80% 
completitud(df)

,columna,total,completitud
0,c_fare_amount,0,100.0
1,d_pickup_datetime,0,100.0
2,c_pickup_longitude,0,100.0
3,c_pickup_latitude,0,100.0
4,c_dropoff_longitude,0,100.0
5,c_dropoff_latitude,0,100.0
6,c_passenger_count,0,100.0


##### Observaciones

En esta ocasión, en el presente dataset, ninguna de las columnas contó con una completitud menor al umbral definido, aunque dos de las variables, **c_dropoff_longitude** y **c_dropoff_latitude**, tenian solo un valor perdido, por lo que se decidió elimimar los registros que presentaban estos valores ausentes.

### Consistencia

#### Fecha

In [74]:
df['d_pickup_datetime'].describe()

count         199999
unique          2372
top       2011-04-27
freq             127
Name: d_pickup_datetime, dtype: object

#### Continuas

In [75]:
#Revisamos las variables categoricas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="c_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

c_fare_amount


array(['-10.9', '-23.7', '-3.0', ..., '98.0', '99.0', '99.2'],
      dtype=object)



c_pickup_longitude


array(['-0.001007', '-0.003813', '-0.005672', ..., '40.806012',
       '40.808425', '57.418457'], dtype=object)



c_pickup_latitude


...

array(['-0.000655', '-0.001178', '-0.001202', ..., '40.828377',
       '40.828672', '40.831932'], dtype=object)



c_dropoff_latitude


array(['-0.000132', '-0.000625', '-0.0022649999999999', ..., '45.031598',
       '493.533332', '872.6976279999999'], dtype=object)



c_passenger_count


array(['0', '1', '2', '208', '3', '4', '5', '6'], dtype=object)

#### Validación

Después de visualizar de forma general las variables continuas, revisemos

**c_passenger_count**
Observamos que hay valores negativos en el precio de los viajes, esto significa que hay valores fuera de la naturaleza de la variable. 

In [76]:
df['c_passenger_count'].value_counts()

1      138425
2       29428
5       14009
3        8881
4        4276
6        4271
0         708
208         1
Name: c_passenger_count, dtype: int64

**c_fare_amount**

Observamos que hay valores negativos en el precio de los viajes, esto significa que hay valores fuera de la naturaleza de la variable. 

In [77]:
(df['c_fare_amount'] < 0).value_counts()

False    199982
True         17
Name: c_fare_amount, dtype: int64

#### Transformación a valores NaN
Al revisar las variables continuas se pueden observar las siguientes variables con inconsistencias y sus respectivas causas:

1. **c_passenger_count**: Esta variable representa el número de pasajeros de cada viaje, normalmente los viajes son en autos con máximo 4 lugares y coches mas grandes con hasta 6 lugares, pero hay datos que se encuentran fuera de la naturaleza de la variables, de modo que requieren reemplzarse con valores NaN, en lugar de solo eliminarlas.
2. **c_fare_amount**: Esta variable representa el número de pasajeros de cada viaje, normalmente los viajes son en autos con máximo 4 lugares y coches mas grandes con hasta 6 lugares, pero hay datos que se encuentran fuera de la naturaleza de la variables, de modo que requieren reemplzarse con valores NaN, en lugar de solo eliminarlas.

In [78]:
#Reemplazamos y validamos
df['c_passenger_count'] = df['c_passenger_count'].where((df['c_passenger_count'] != 0) & (df['c_passenger_count'] != 208), np.nan)
display(df['c_passenger_count'].value_counts())

#Reemplazamos y validamos
df['c_fare_amount'] = df['c_fare_amount'].where(df['c_fare_amount'] > 0, np.nan)
display((df['c_fare_amount'] < 0).value_counts())

1.0    138425
2.0     29428
5.0     14009
3.0      8881
4.0      4276
6.0      4271
Name: c_passenger_count, dtype: int64

False    199999
Name: c_fare_amount, dtype: int64

### Completitud

Se recomienda que las variables cuenten con una completitud de al menos 80%, de modo que las que no cumplan con esta condición serán eliminadas.

In [42]:
df.isnull().sum(0)

c_fare_amount           17
d_pickup_datetime        0
c_pickup_longitude       0
c_pickup_latitude        0
c_dropoff_longitude      0
c_dropoff_latitude       0
c_passenger_count      709
dtype: int64

In [43]:
#Revisamos la completitud de las columnas del dataset y observamos que aunque no hay peridda de valores menor al 80%, ahora tenemos una variables con 709 valores faltantes
completitud(df)

,columna,total,completitud
0,c_passenger_count,709,99.645498
1,c_fare_amount,17,99.991500
2,d_pickup_datetime,0,100.000000
3,c_pickup_longitude,0,100.000000
4,c_pickup_latitude,0,100.000000
5,c_dropoff_longitude,0,100.000000
6,c_dropoff_latitude,0,100.000000


## Exploratory Data Analysis (EDA)

In [84]:
nu = df.drop(["c_fare_amount"], axis=1).nunique().sort_values()
nu

c_passenger_count          6
d_pickup_datetime       2372
c_pickup_longitude     71065
c_dropoff_longitude    76894
c_pickup_latitude      83835
c_dropoff_latitude     90585
dtype: int64

In [83]:
#Checking number of unique rows in each feature

nu = df.drop(["c_fare_amount"], axis=1).nunique().sort_values()
nf = []; cf = []; nnf = 0; ncf = 0; #numerical & categorical features

for i in range(df.drop(["c_fare_amount"], axis=1).shape[1]):
    if nu.values[i]<=24:cf.append(nu.index[i])
    else: nf.append(nu.index[i])

print('\n\033[1mInference:\033[0m The Datset has {} numerical & {} categorical features.'.format(len(nf),len(cf)))


Inference: The Datset has 5 numerical & 1 categorical features.


In [47]:
#plt.figure(figsize=[15,10])
a=plt.imread('https://raw.githubusercontent.com/Masterx-AI/Project_Uber_Fare_Prediction/main/wm.png')
#plt.imshow(a, alpha=0.2)
#plt.scatter( (df.pickup_longitude+180)*3,(df.pickup_latitude+215)*1.45555555,alpha=0.3, color='red')
#mdf.plot(kind='scatter',x='pickup_latitude',y='pickup_longitude',alpha=0.1)
#plt.show()

ValueError: Please open the URL for reading and pass the result to Pillow, e.g. with ``np.array(PIL.Image.open(urllib.request.urlopen(url)))``.

In [79]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
c_fare_amount,199977.0,11.362586,9.897063,0.010000,6.000000,8.500000,12.500000,499.000000
c_pickup_longitude,199999.0,-72.527631,11.437815,-1340.648410,-73.992065,-73.981823,-73.967154,57.418457
c_pickup_latitude,199999.0,39.935881,7.720558,-74.015515,40.734796,40.752592,40.767158,1644.421482
c_dropoff_longitude,199999.0,-72.525292,13.117408,-3356.666300,-73.991407,-73.980093,-73.963658,1153.572603
c_dropoff_latitude,199999.0,39.923890,6.794829,-881.985513,40.733823,40.753042,40.768001,872.697628
c_passenger_count,199290.0,1.689493,1.305420,1.000000,1.000000,1.000000,2.000000,6.000000
